# Lab 2: A Brief Introduction to MLflow 

## Goal
Setup the MLflow, and get familiar with the basics.
Inspired by the [hands-on tutorial from MLflow](https://mlflow.org/docs/latest/ml/getting-started/logging-first-model/).

## About MLflow

MLflow is an open-source platform designed to manage the machine learning lifecycle, including:

* **Experiment tracking:** log parameters, metrics, artifacts (models, plots, datasets).

* **Model packaging:** save models in a standard format for deployment.

* **Model registry:** register, version, and stage models (e.g., “Staging” or “Production”).

* **Deployment:** serve models via REST APIs or integrate with other platforms.


MLflow supports multiple ways to track experiments and runs:

* Local file system (default)

    * Stores run data under mlruns/ in the project directory.

    * Easy for experimentation on a single machine.

* MLflow Tracking Server

    * Centralized server for multiple users.

    * Can be backed by a database (e.g., MySQL, PostgreSQL) for runs metadata.

    * Artifacts can be stored in cloud storage (S3, Azure Blob, GCS) or network drives.

* Cloud-hosted services

    * Managed platforms like Databricks, AWS SageMaker, or GCP AI Platform integrate MLflow tracking.

    * Provide collaboration, access control, and remote artifact storage.

## Part 1: Setup & Foundations

In [ ]:
# install dependencies
!pip install 'mlflow[extras]<3' hyperopt tensorflow scikit-learn pandas numpy

Open a console and launch a MLflow tracking server:

```
mlflow server --host 127.0.0.1 --port 8080
```

## Part 2: Data & Experiments

In [ ]:
import mlflow
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Sets the current active experiment to the "Iris_Models" experiment and
# returns the Experiment metadata
iris_experiment = mlflow.set_experiment("Iris_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "iris_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_iris"

# Load the data and split it into training and validation sets
iris = load_iris()
X_train, X_val, y_train, y_val = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

params = {
    "n_estimators": 50,
    "max_depth": 3,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, artifact_path=artifact_path)

In order to see the results of our run, we can navigate to the MLflow UI. Since we have already started the Tracking Server at http://localhost:8080, we can simply navigate to that URL in our browser.

### Analyze Results in MLflow UI
In order to see the results of our run, we can navigate to the MLflow UI. Since we have already started the Tracking Server at http://localhost:8080, we can simply navigate to that URL in our browser.

You can also run following command in a separate terminal and keep it running:
```
mlflow ui --port 5000
```
Your MLflow UI will be available at http://localhost:5000.

## Part 3: Hyperparameter Tuning & Deployment

### Hyperparameter Tuning

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK


experiment_name = "iris-rf-optimization"
mlflow.set_experiment(experiment_name)

print(f"Starting hyperparameter optimization experiment: {experiment_name}")
print("This will run 15 trials to find optimal hyperparameters...")

# Define search space
search_space = {
    "n_estimators": hp.choice("n_estimators", [10, 50, 100]),
    "max_depth": hp.choice("max_depth", [3, 5, 10]),
}

# Define objective function
def objective(params):
    with mlflow.start_run(nested=True):
        # Log parameters
        mlflow.log_params(params)

        # Train model
        model = RandomForestRegressor(
            n_estimators=int(params["n_estimators"]),
            max_depth=params["max_depth"],
            random_state=42,
        )
        model.fit(X_train, y_train)

        # Evaluate
        y_pred = model.predict(X_val)
        # Calculate error metrics
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_val, y_pred)

        # Assemble the metrics we're going to write into a collection
        metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

        # Log metrics
        mlflow.log_metrics(metrics)

        # Log model
        mlflow.sklearn.log_model(model, input_example=X_val, artifact_path="model")

    return {"loss": rmse, "status": STATUS_OK}

# Run optimization
trials = Trials()

with mlflow.start_run(run_name="hyperparameter-sweep"):
    mlflow.log_params(
        {
            "max_evaluations": 15,
            "objective_metric": "rmse",
            "dataset": "iris",
            "model_type": "RandomForestRegressor",
        }
    )

    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=15,
        trials=trials,
        verbose=True,
    )

    best_trial = min(trials.results, key=lambda x: x["loss"])
    best_rmse = best_trial["loss"]

    mlflow.log_params(
        {
            "best_n_estimators": best_params["n_estimators"],
            "best_max_depth": best_params["max_depth"],
        }
    )

    mlflow.log_metrics(
        {
            "best_rmse": best_rmse,
            "total_trials": len(trials.trials),
            "optimization_completed": 1,
        }
    )

print(f"Best parameters: {best_params}")
print(f"Best RMSE: {best_rmse:.4f}")

### Register Your Best Model & Deploy It Locally

In [ ]:
# We can either use the UI to find and register the best model or do it via Python:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("iris-rf-optimization")
runs = client.search_runs(experiment.experiment_id, order_by=["metrics.rmse ASC"])
best_run = runs[0]

print("Best run ID:", best_run.info.run_id)
print("RMSE:", best_run.data.metrics["rmse"])

# Load model from MLflow
model_uri = f"runs:/{best_run.info.run_id}/model"
best_model = mlflow.sklearn.load_model(model_uri)

# Test prediction
sample = X_val[0].reshape(1, -1)
prediction = best_model.predict(sample)
print(f"Prediction: {prediction[0]}")
print(f"Ground truth: {y_val[0]}")

# Register best model
model_name = "Iris-RandomForest"
print(f"Registering best model: {model_name}")
registered_model = mlflow.register_model(model_uri, model_name)

print(f"Model registered: {registered_model.name}, version {registered_model.version}")

In [ ]:
# Serve the model locally (choose the version number you registered) - run in a separate terminal:
export MLFLOW_TRACKING_URI=http://localhost:8080
mlflow models serve -m "models:/Iris-RandomForest/1" -p 5002 --env-manager conda

In [ ]:
# We can then test the deployment with Python
import requests
import json


data = json.dumps({"instances": X_val[:2].tolist()})
headers = {"Content-Type": "application/json"}

response = requests.post("http://127.0.0.1:5002/invocations", data=data, headers=headers)
print("Predictions:", response.json())

In [ ]:
# We can also build a Docker image for cloud deployment
mlflow models build-docker --model-uri "models:/Iris-RandomForest/1" --name "iris-rf-api"